In [1]:
#%pip install mysqlclient
#%pip install configparser
#%pip install pyecharts

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dataDealer import getData 
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output

df = getData()
columns = {"MaxPlayTime" : 'MaxPlayTime', 
           "MinPlayTime" : 'MinPlayTime',
           "MaxPlayers" : 'MaxPlayers',
           "MinPlayers" : 'MinPlayers',
           "BestPlayers" : 'BestPlayers',
           "MinAge" : 'MinAge',
           "Weight" : 'Weight',
           "Year":'Year'
          }


@interact
def show_articles_more_than(column= columns, x=(0,500)): 
    print(f"符合條件的桌遊有 {len(df.loc[df[column] > x])} 款")
    return df.loc[df[column] > x]


interactive(children=(Dropdown(description='column', options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTime': 'Mi…

In [2]:
def update_annot(ind):
    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    tempList = ind["ind"]
    text = ydf['Name'][int(tempList)]
    annot.set_text(text)



def hover(event):
    
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()

                
@interact(col1 = columns, col2 = columns)
def setData(col1= 'Year', col2= 'Weight'):
    global ydf
    ydf = df.sort_values([col1]).reset_index(drop=True)
    
    x = ydf[col1]
    y = ydf[col2]

    global ax
    fig,ax = plt.subplots()
    global sc
    sc = plt.scatter(x, y)
    global annot
    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                  bbox=dict(boxstyle="round", fc="w"),
                   )
    annot.set_visible(False)
    
    fig.canvas.mpl_connect("motion_notify_event", hover)

    plt.show()



interactive(children=(Dropdown(description='col1', index=7, options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTim…

In [3]:
from pyecharts import options as opts
from pyecharts.charts import Pie

@interact()
def rosePie(col1 = columns):
    data = df[col1].value_counts()
    v = data.index
    c = (
        Pie()
        .add(
            "123",
            [list(z) for z in zip(v, list(data))],
            radius=["50%", "80%"],
            center=["50%", "50%"],
            rosetype="area",
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="玫瑰圖展示"))

    )
    return c.render_notebook()
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 40em; }</style>"))

interactive(children=(Dropdown(description='col1', options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTime': 'MinP…

In [4]:
import pyecharts.options as opts
from pyecharts.charts import Bar3D

df2 = df.groupby(['MinPlayers','MaxPlayTime']).size()
nlist = [list(df2.index[i])+[df2.values[i]] for i in range(len(df2))]
maxTime = [
    20,
    30,
    45,
    60,
    70,
    80,
    90,
    100,
    115,
    120,
    150,
    180,
    200,
    210,
    240,
    480,
    1000
]


minPlayer = [1, 2, 3]
data = [[minPlayer.index(item[0]),maxTime.index(item[1]),item[2]] for item in nlist]
data = [[d[1], d[0], int(d[2])] for d in data]

#print(type(data[0][2]))

(
    Bar3D()
    .add(
        series_name="",
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(type_="category", data=maxTime),
        yaxis3d_opts=opts.Axis3DOpts(type_="category", data=minPlayer),
        zaxis3d_opts=opts.Axis3DOpts(type_="value"),
    )
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(
            max_=8,
            range_color=[
                "#313695",
                "#4575b4",
                "#74add1",
                "#abd9e9",
                "#e0f3f8",
                "#ffffbf",
                "#fee090",
                "#fdae61",
                "#f46d43",
                "#d73027",
                "#a50026",
            ],
        )
    )
    .render_notebook()
)